<a href="https://colab.research.google.com/github/88FaHaD/ML/blob/main/effinetdetectorbirds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf


In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3074/3074), done.
remote: Total 4084 (delta 1189), reused 2904 (delta 950), pack-reused 0
Receiving objects: 100% (4084/4084), 44.61 MiB | 23.22 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━

In [4]:
#run model builder test
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2024-03-06 09:22:04.382813: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 09:22:04.382916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 09:22:04.385108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 09:22:06.255254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0306 09:22:11.442495 133472492793856 batch_normalization.py:1531] `tf.keras.layers.experimental.SyncBatchNormalization` end

In [5]:
import pandas as pd

In [6]:
! pip install kaggle

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
! mkdir ~/.kaggle

In [9]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download gpiosenka/100-bird-species

100% 1.95G/1.96G [01:07<00:00, 32.9MB/s]
100% 1.96G/1.96G [01:07<00:00, 31.2MB/s]


In [13]:
import zipfile

# Specify the path to the zip file
zip_file_path = "/content/100-bird-species.zip"

# Specify the directory where you want to extract the contents
extract_to_directory = "/content/bird_species_data"

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)


In [17]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (160, 160)
BATCH_SIZE = 32

train_dir = '/content/bird_species_data/train'
valid_dir = '/content/bird_species_data/valid'
test_dir=   '/content/bird_species_data/test'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("validdation images:")
valid_data = train_datagen.flow_from_directory(valid_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")



Training images:
Found 84635 images belonging to 525 classes.
validdation images:
Found 2625 images belonging to 525 classes.


In [18]:
labels='/content/bird_species_data/birds.csv'


In [19]:
batch_size = 16
num_steps = 8000
num_eval_steps = 1000

In [20]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz


--2024-03-06 09:43:41--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.23.207, 74.125.203.207, 64.233.187.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.23.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M  16.2MB/s    in 1.8s    

2024-03-06 09:43:44 (16.2 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]



In [21]:
fine_tune_checkpoint = 'efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'

In [25]:

!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config

base_config_path = 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

--2024-03-06 10:05:17--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4630 (4.5K) [text/plain]
Saving to: ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config.1’

ssd_efficientdet_d0 100%[===================>]   4.52K  --.-KB/s    in 0s      

2024-03-06 10:05:17 (36.8 MB/s) - ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config.1’ saved [4630/4630]



In [26]:
import re

# Set paths
base_config_path = 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config'
label_map_path = '/content/bird_species_data/label_map.pbtxt'
fine_tune_checkpoint = '/content/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'
train_record_path = '/content/bird_species_data/train.record'
test_record_path = '/content/bird_species_data/valid.record'
num_classes = 524  # Adjust according to your data
batch_size = 16
num_steps = 8000
num_eval_steps = 1000

# Read base configuration file
with open(base_config_path) as f:
    config = f.read()

# Edit configuration
config = re.sub('label_map_path: ".*?"',
                'label_map_path: "{}"'.format(label_map_path), config)

config = re.sub('fine_tune_checkpoint: ".*?"',
                'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                'input_path: "{}"'.format(train_record_path), config)

config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
                'input_path: "{}"'.format(test_record_path), config)

config = re.sub('num_classes: [0-9]+',
                'num_classes: {}'.format(num_classes), config)

config = re.sub('batch_size: [0-9]+',
                'batch_size: {}'.format(batch_size), config)

config = re.sub('num_steps: [0-9]+',
                'num_steps: {}'.format(num_steps), config)

config = re.sub('fine_tune_checkpoint_type: "classification"',
                'fine_tune_checkpoint_type: "{}"'.format('detection'), config)

# Write modified configuration to a new file
with open('custom_model_config.config', 'w') as f:
    f.write(config)

# Start training
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=custom_model_config.config \
    --model_dir=training/ \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

# Export trained model
output_directory = '/content/inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir=training/ \
    --output_directory={output_directory} \
    --pipeline_config_path=custom_model_config.config


2024-03-06 10:09:11.199421: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 10:09:11.199502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 10:09:11.202543: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 10:09:12.729650: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0306 10:09:18.252739 134996648366080 mirrored_strategy.py:423] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/devic

In [30]:
output_directory = '/content/models'


In [31]:
# Export trained model
output_directory = '/content/models'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir=training/ \
    --output_directory={output_directory} \
    --pipeline_config_path=custom_model_config.config

# Save model
model_path = output_directory


2024-03-06 10:28:05.504072: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 10:28:05.504143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 10:28:05.506316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 10:28:07.037261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0306 10:28:11.555752 138190805454848 ssd_efficientnet_bifpn_feature_extractor.py:161] EfficientDet EfficientNet backbone version: efficientnet-b0
I0306 10:28:11.556889 138190805454848 ssd_efficientnet_bifpn_feature_extractor.py:163] Efficie